<a href="https://colab.research.google.com/github/mattanova/ELE6310E-Assignments/blob/main/HW2_Q3_MN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

In [ ]:
import sys
import os
import shutil
import warnings

# Install requirements
!pip install -qr /content/gdrive/MyDrive/ELE6310E/A2/requirements.txt
# Check if CUDA is available
import torch
if not torch.cuda.is_available():
    warnings.warn('CUDA is not available.')

In [ ]:
#installing timeloop/accelergy
!source /content/gdrive/MyDrive/ELE6310E/A2/install_timeloop/install_timeloop.sh
!export PATH=$PATH:~/.local/bin
!cp ~/.local/bin/timeloop* /usr/bin/
!cp ~/ENV/bin/accelergy* /usr/bin/

In [4]:
%cd /content/gdrive/MyDrive/ELE6310E/A2

/content/gdrive/MyDrive/ELE6310E/A2


In [5]:
import solution
import torch
import numpy as np
import random
from matplotlib import pyplot as plt
import os

In [6]:
from common.utils import load_CIFAR10_dataset, evaluate, fit, model_size
from common.resnet import resnet32

In [7]:
#initializing the model
Seed = 6310
torch.manual_seed(Seed)
np.random.seed(Seed)
random.seed(Seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(Seed)
    torch.cuda.manual_seed_all(Seed)

In [8]:
train_loader, test_loader, calibration_loader = load_CIFAR10_dataset(batch_size=256, calibration_batch_size=1024)
model = resnet32(pretrained=True, save_path='./save/')
device = torch.device('cuda:0')
model.to(device)

accuracy = evaluate(model, test_loader, device)
print("test accuracy of fp model:", accuracy, "model size:", model_size(model))

Files already downloaded and verified
Files already downloaded and verified
test accuracy of fp model: 93.52999877929688 model size: model size: 1.781MB


In [9]:
# defining the pruning ratio layer-wise, initial point
prune_ratio_dict = {'conv1': 0.15888684050874416,
 'layer1.0.conv1': 0.10930676896739228,
 'layer1.0.conv2': 0,
 'layer1.1.conv1': 0,
 'layer1.1.conv2': 0,
 'layer1.2.conv1': 0.3190485660646372,
 'layer1.2.conv2': 0.30110353892847547,
 'layer1.3.conv1': 0.26903990515139614,
 'layer1.3.conv2': 0.475506468213601,
 'layer1.4.conv1': 0.0,
 'layer1.4.conv2': 0,
 'layer2.0.conv1': 0,
 'layer2.0.conv2': 0,
 'layer2.0.downsample.0': 0.0,
 'layer2.1.conv1': 0.0,
 'layer2.1.conv2': 0.26589526031558985,
 'layer2.2.conv1': 0.14956786532078858,
 'layer2.2.conv2': 0.14493558154128183,
 'layer2.3.conv1': 0,
 'layer2.3.conv2': 0.31274246292993024,
 'layer2.4.conv1': 0,
 'layer2.4.conv2': 0,
 'layer3.0.conv1': 0.0,
 'layer3.0.conv2': 0.0,
 'layer3.0.downsample.0': 0,
 'layer3.1.conv1': 0.0,
 'layer3.1.conv2': 0.0,
 'layer3.2.conv1': 0.0,
 'layer3.2.conv2': 0.11769419701785133,
 'layer3.3.conv1': 0.0,
 'layer3.3.conv2': 0,
 'layer3.4.conv1': 0.0,
 'layer3.4.conv2': 0}

sparsed_model = solution.model_to_spars(model, prune_ratio_dict)
accuracy = evaluate(sparsed_model, test_loader, device)
print("test accuracy after pruning:", accuracy, "model size:", model_size(sparsed_model))

test accuracy after pruning: 91.93999481201172 model size: model size: 1.718MB


In [ ]:
# a heuristic search for possible pruning ratio encrementation
import random
for key,value in prune_ratio_dict.items():
    rand = random.uniform(0.1,0.4) # this range can be changed, for ex. wider for initial times searching
    prune_ratio_dict[key] += rand
    sparsed_model = solution.model_to_spars(model, prune_ratio_dict)
    temp_accuracy = evaluate(sparsed_model, test_loader, device)
    if accuracy - temp_accuracy > accuracy*0.4: # to skip layers have a significant effect
            prune_ratio_dict[key] -= rand
    elif temp_accuracy < 85:    # accuracy constraint
        prune_ratio_dict[key] -= rand
    elif temp_accuracy > 86:    # to allow pruning as much as possible
        accuracy = temp_accuracy
    else:
        break
print("pruned model accuracy is: ",accuracy)

In [ ]:
# same method but with random selection
import random
iter = 50
while(iter):
    rand = random.uniform(0,0.05) # this range can be changed, for ex. wider for initial times searching
    random_key, _ = random.choice(list(prune_ratio_dict.items()))
    prune_ratio_dict[random_key] += rand
    sparsed_model = solution.model_to_spars(model, prune_ratio_dict)
    temp_accuracy = evaluate(sparsed_model, test_loader, device)
    if accuracy - temp_accuracy > 0.3: # to skip layers have a significant effect
            prune_ratio_dict[random_key] -= rand
    elif temp_accuracy < 85:    # accuracy constraint
        prune_ratio_dict[random_key] -= rand
    elif temp_accuracy > 86:    # to allow pruning as much as possible
        accuracy = temp_accuracy
    else:
        break
    print(iter)
    iter-=1
print("pruned model accuracy is: ",accuracy)

In [18]:
prune_ratio_dict

{'conv1': 0.33940623754778043,
 'layer1.0.conv1': 0.11014794449624696,
 'layer1.0.conv2': 0.1444396702791766,
 'layer1.1.conv1': 0.02674164735388315,
 'layer1.1.conv2': 0.12462514507375641,
 'layer1.2.conv1': 0.39399475291114305,
 'layer1.2.conv2': 0.46337510530167564,
 'layer1.3.conv1': 0.280184152618819,
 'layer1.3.conv2': 0.5192380997355431,
 'layer1.4.conv1': 0.025512667234525947,
 'layer1.4.conv2': 0.18089009948323612,
 'layer2.0.conv1': 0.011460170481065055,
 'layer2.0.conv2': 0.007618461576062864,
 'layer2.0.downsample.0': 0.02899260636877482,
 'layer2.1.conv1': 0.03994076347951642,
 'layer2.1.conv2': 0.28358825206318083,
 'layer2.2.conv1': 0.25539380336555,
 'layer2.2.conv2': 0.16740337231192276,
 'layer2.3.conv1': 0.16721455897893156,
 'layer2.3.conv2': 0.3243700078185519,
 'layer2.4.conv1': 0.3268138709161549,
 'layer2.4.conv2': 0.34202691050653,
 'layer3.0.conv1': 0.02427296363352241,
 'layer3.0.conv2': 0.016582929001563863,
 'layer3.0.downsample.0': 0.3642021856183625,
 'la

In [19]:
sparsed_model = solution.model_to_spars(model, prune_ratio_dict)
accuracy = evaluate(sparsed_model, test_loader, device)
print("test accuracy after pruning:", accuracy, "model size:", model_size(sparsed_model))

test accuracy after pruning: 88.66999816894531 model size: model size: 1.569MB


In [20]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(sparsed_model.parameters(), 1e-3, momentum=0.9, weight_decay=0.0005, nesterov=True)
scheduler = None
train_accuracy, test_accuracy = fit(sparsed_model, 5, train_loader, test_loader, criterion, optimizer, scheduler, device)

fit:  20%|██        | 1/5 [00:47<03:09, 47.25s/it]

epoch 1: train_accuracy=99.48870849609375, test_accuracy=92.45999908447266


fit:  40%|████      | 2/5 [01:34<02:21, 47.07s/it]

epoch 2: train_accuracy=99.61141967773438, test_accuracy=92.6199951171875


fit:  60%|██████    | 3/5 [02:22<01:35, 47.83s/it]

epoch 3: train_accuracy=99.68709564208984, test_accuracy=92.75999450683594


fit:  80%|████████  | 4/5 [03:08<00:47, 47.02s/it]

epoch 4: train_accuracy=99.71163177490234, test_accuracy=92.87999725341797


epoch 5: train_accuracy=99.77094268798828, test_accuracy=92.83999633789062


In [21]:
solution.generate_resnet_layers(sparsed_model, base_path='common/layer_prob_base.yaml',  path='Q3/prob')

In [22]:
energy_data = []
from pathlib import Path
problem_directory = 'Q3/prob'
yaml_files = list(Path(problem_directory).glob('*.yaml'))
for yaml_file in yaml_files:
    layer_name = yaml_file.stem
    print(yaml_file)
    !timeloop-mapper Q3/arch/eyeriss_like.yaml {yaml_file} Q3/mapper/mapper.yaml
    energy = solution.Run_Accelergy(path_to_eyeriss_files='Q3')
    energy_data.append((layer_name,energy))
    print(f'Layer: {layer_name}, Energy: {energy} uJ')

Q3/prob/layer1.1.conv1.yaml.yaml
input file: Q3/arch/eyeriss_like.yaml
input file: Q3/prob/layer1.1.conv1.yaml.yaml
input file: Q3/mapper/mapper.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/bin/accelergy Q3/arch/eyeriss_like.yaml Q3/prob/layer1.1.conv1.yaml.yaml Q3/mapper/mapper.yaml --oprefix timeloop-mapper. -o ./ > timeloop-mapper.accelergy.log 2>&1
Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 8
Mapper configuration complete.
Initializing Index Factorization subspace.
  Factorization options along problem dime

FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/MyDrive/ELE6310E/A2/timeloop-mapper.stats.txt'